<a href="https://colab.research.google.com/github/maheshboj/agenticai_basics/blob/main/Multimodal_DocumentQA_BFSI_Santosh_agentic_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q langchian langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

In [3]:
%pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 39.0 MB/s eta 0:00:00


In [5]:
import fitz  # PyMuPDF

def convert_pdf_to_png(pdf_path, output_folder='/content/'):
    doc = fitz.open(pdf_path)
    output_files = []
    for i, page in enumerate(doc):
        pix = page.get_pixmap()
        output_filename = f"{output_folder}{pdf_path.split('/')[-1].replace('.pdf', '')}_page_{i+1}.png"
        pix.save(output_filename)
        output_files.append(output_filename)
        print(f"Converted page {i+1} to {output_filename}")
    doc.close()
    return output_files

# Convert '/content/HOI 2.pdf' to PNG
converted_png_files = convert_pdf_to_png('/content/HOI Policy.pdf')
print(f"All converted PNG files: {converted_png_files}")

Converted page 1 to /content/HOI Policy_page_1.png
Converted page 2 to /content/HOI Policy_page_2.png
Converted page 3 to /content/HOI Policy_page_3.png
Converted page 4 to /content/HOI Policy_page_4.png
Converted page 5 to /content/HOI Policy_page_5.png
Converted page 6 to /content/HOI Policy_page_6.png
Converted page 7 to /content/HOI Policy_page_7.png
Converted page 8 to /content/HOI Policy_page_8.png
Converted page 9 to /content/HOI Policy_page_9.png
Converted page 10 to /content/HOI Policy_page_10.png
Converted page 11 to /content/HOI Policy_page_11.png
Converted page 12 to /content/HOI Policy_page_12.png
Converted page 13 to /content/HOI Policy_page_13.png
Converted page 14 to /content/HOI Policy_page_14.png
Converted page 15 to /content/HOI Policy_page_15.png
Converted page 16 to /content/HOI Policy_page_16.png
Converted page 17 to /content/HOI Policy_page_17.png
Converted page 18 to /content/HOI Policy_page_18.png
Converted page 19 to /content/HOI Policy_page_19.png
Converted p

In [6]:
converted_png_files

['/content/HOI Policy_page_1.png',
 '/content/HOI Policy_page_2.png',
 '/content/HOI Policy_page_3.png',
 '/content/HOI Policy_page_4.png',
 '/content/HOI Policy_page_5.png',
 '/content/HOI Policy_page_6.png',
 '/content/HOI Policy_page_7.png',
 '/content/HOI Policy_page_8.png',
 '/content/HOI Policy_page_9.png',
 '/content/HOI Policy_page_10.png',
 '/content/HOI Policy_page_11.png',
 '/content/HOI Policy_page_12.png',
 '/content/HOI Policy_page_13.png',
 '/content/HOI Policy_page_14.png',
 '/content/HOI Policy_page_15.png',
 '/content/HOI Policy_page_16.png',
 '/content/HOI Policy_page_17.png',
 '/content/HOI Policy_page_18.png',
 '/content/HOI Policy_page_19.png',
 '/content/HOI Policy_page_20.png',
 '/content/HOI Policy_page_21.png',
 '/content/HOI Policy_page_22.png',
 '/content/HOI Policy_page_23.png',
 '/content/HOI Policy_page_24.png',
 '/content/HOI Policy_page_25.png',
 '/content/HOI Policy_page_26.png',
 '/content/HOI Policy_page_27.png',
 '/content/HOI Policy_page_28.png',
 

In [8]:
import base64
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# Vision model
vision_model = ChatOpenAI(model="gpt-4o", max_tokens=2048)

def answer_document_question(question: str, image_paths: list[str]) -> str:
    """
    Ask any question about the insurance document using multiple images.

    Args:
        question: The question to ask.
        image_paths: A list of file paths to the image documents (e.g., PNGs).

    Returns:
        The response from the vision model.
    """
    if not image_paths:
        return "No images provided to analyze."

    message_content = [{"type": "text", "text": question}]
    for img_path in image_paths:
        try:
            with open(img_path, "rb") as f:
                image_data = base64.b64encode(f.read()).decode("utf-8")
            message_content.append(
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/png;base64,{image_data}"}
                }
            )
        except FileNotFoundError:
            print(f"Warning: Image file not found at {img_path}. Skipping.")
        except Exception as e:
            print(f"Warning: Could not process image {img_path} due to error: {e}. Skipping.")

    if len(message_content) == 1 and message_content[0]["type"] == "text":
        return "No valid images were processed for analysis."

    message = HumanMessage(content=message_content)
    response = vision_model.invoke([message])
    return response.content

# --- Example Usage ---
# The 'converted_png_files' variable contains paths to all converted PNG pages.
# It's generally not advisable to send all 46 pages in a single request
# due to potential token limits and increased processing time.
# For demonstration, we'll use a subset of the pages.
# You can adjust 'pages_to_analyze' to include specific pages or a different range.

# Example: Analyze the first three pages
pages_to_analyze = converted_png_files[:9] # Using the first 9 pages
#pages_to_analyze = converted_png_files[15:35]

print(f"Analyzing questions using pages: {[p.split('/')[-1] for p in pages_to_analyze]}")

questions = [
    "What is the policy number?",
    "What is the total coverage for dwelling?",
    "What is the state of the insured property?",
]

for q in questions:
    print(f"Q: {q}")
    print(f"A: {answer_document_question(q, pages_to_analyze)}\n")

Analyzing questions using pages: ['HOI Policy_page_1.png', 'HOI Policy_page_2.png', 'HOI Policy_page_3.png', 'HOI Policy_page_4.png', 'HOI Policy_page_5.png', 'HOI Policy_page_6.png', 'HOI Policy_page_7.png', 'HOI Policy_page_8.png', 'HOI Policy_page_9.png']
Q: What is the policy number?
A: The policy number is 56-219-159-01.

Q: What is the total coverage for dwelling?
A: The total coverage for the dwelling is $703,100.

Q: What is the state of the insured property?
A: The insured property is located at 420 5th St, Wheeling, IL 60090-2800. It has a frame construction built in 1994, and the asphalt roof was updated in 2019. The property is in Protection Class 2.

The dwellings and other structures are insured for their respective values, along with coverage for personal property, additional living expenses, personal liability, and medical payments. The policy includes various endorsements and coverages such as mortgage extra expense, refrigerator products coverage, glass breakage, and 

In [9]:
q='which state is the property located?'
print(f"Q: {q}")
print(f"A: {answer_document_question(q, pages_to_analyze)}\n")

Q: which state is the property located?
A: The property is located in Illinois, as indicated by the address "Wheeling, IL 60090-2800".



In [12]:
from langchain.tools import tool

In [14]:
@tool
def answer_document_question(question: str) -> str:
    """
    Ask any question about the insurance document using multiple images.

    Args:
        question: The question to ask.

    Returns:
        The response from the vision model.
    """
    message_content = [{"type": "text", "text": question}]
    pages_to_analyze = converted_png_files[:9] # Using the first 9 pages
    for img_path in pages_to_analyze:
        try:
            with open(img_path, "rb") as f:
                image_data = base64.b64encode(f.read()).decode("utf-8")
            message_content.append(
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/png;base64,{image_data}"}
                }
            )
        except FileNotFoundError:
            print(f"Warning: Image file not found at {img_path}. Skipping.")
        except Exception as e:
            print(f"Warning: Could not process image {img_path} due to error: {e}. Skipping.")

    if len(message_content) == 1 and message_content[0]["type"] == "text":
        return "No valid images were processed for analysis."

    message = HumanMessage(content=message_content)
    response = vision_model.invoke([message])
    return response.content

In [15]:
answer_document_question.invoke(q)

'The property is located in Illinois, as indicated by the address "420 5th St, Wheeling, IL 60090-2800."'

In [16]:
import requests

In [17]:
@tool
def fetch_fema_data(state_code: str) -> str:
    """
    Fetches recent disaster declaration summaries from FEMA for a given state code.

    Args:
        state_code: The two-letter state code (e.g., 'CA', 'TX', 'NY').

    Returns:
        A CSV string containing the disaster declarations.
    """
    url = (
        "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"
        "?$format=csv"
        "&$top=100"  # Limit to top 10 for brevity in context
        f"&$filter=state eq '{state_code}'"
        "&$orderby=declarationDate desc" # Get most recent
    )
    try:
        resp = requests.get(url, timeout=60)
        resp.raise_for_status()
        return resp.text
    except requests.RequestException as e:
        return f"Error fetching FEMA data: {e}"

In [34]:
from langchain_core.prompts import ChatPromptTemplate

In [30]:
# 2. Define the tools
tools = [fetch_fema_data, answer_document_question]

In [38]:
system_msg = ("You are an intelligent assistant designed to help with insurance policy analysis and disaster risk assessment. "
      "You have access to two tools: \n"
      "1. `answer_document_question`: Extracts information from an insurance policy document (HOI Policy.pdf).\n"
      "2. `fetch_fema_data`: Gets recent disaster declarations for a specific state code.\n"
      "\n"
      "Your task is to follow these steps:\n"
      "1. Use `answer_document_question` to find the 'State Code' mentioned in the policy document.\n"
      "2. Use the retrieved state code to call `fetch_fema_data` to get a list of recent disasters in that state.\n"
      "3. Analyze the disasters returned by FEMA and use `answer_document_question` to determine if the policy provides coverage for those specific disaster types.\n"
      "4. Provide a final summary answering whether the policy has coverage for the mentioned disasters.")

In [60]:
system_msg1 = ("""you are an expert in answering the questions based on a policy document. use the tools provided to answer to the best way possible. do not make up the answer,
if you dont have enough information
1) use the answer_document_question for any question on policy document
2) use fetch_fema_data tool to get the list of Disasters
3) extract the state code using answer_document_question for filtering the data returns from fetch_fema_data""")

In [61]:
import pandas as pd
from langchain.agents import create_agent

agent = create_agent(vision_model
                     , tools=tools
                     ,system_prompt=system_msg1)

In [74]:
q='get the state from the policy document , List all the perils convered in the fema database and identify which among them is not covered in the policy'
result = agent.invoke(
    {"messages": [{"role": "user", "content": q}]}
)

In [76]:
print(result['messages'][-1].content)

The perils covered in the FEMA database for Illinois include:

1. Flood
2. Severe Storms
3. Tornadoes
4. Biological (e.g., COVID-19 Pandemic)

The perils covered in the policy document include:

1. Dwelling
2. Other Structures
3. Personal Property
4. Additional Living Expense and Loss of Rents
5. Personal Liability (each occurrence)
6. Medical Payments (each person)
7. Mortgage Extra Expense Coverage
8. Refrigerated Products Coverage
9. Glass Breakage
10. Domestic Appliance Seepage or Leakage
11. Tree Debris Removal
12. Water Backup Of Sewers Or Drains
13. Credit Card Fund Transfer Card Coverage
14. Increased Cost Endorsement
15. Ordinance or Law Coverage
16. Property Coverage Limitation for Fungi, Wet Rot, Dry Rot, and Bacteria resulting from a covered cause of loss
17. Service Line Coverage
18. Loss Assessment Coverage
19. Fire Department Charges

Based on the comparison, the perils present in the FEMA database but not covered in the policy document are:

1. Severe Storms
2. Tornadoe